## 12.7 Subsets and permutations

As promised in the previous chapter, I now show two algorithms
to generate permutations and subsets. Both are recursive.
The main aims of this section are
to illustrate more complicated combination steps and show that the decrease step
splits the input collection into an item and the rest of the collection.
Any item will do, but for sequences it's usually the head.

### 12.7.1 Generating subsets

Let's generate all subsets of a given set of items. The output will be
a sequence of subsets so that we can see in which order they're generated.
The usual two questions, adapted to this problem, are:

1. What's the smallest input and the corresponding output?
   It's the empty set; the output is a sequence just with the empty set.

2. If we have all the subsets for a set with one member less,
   how do we obtain the subsets of the whole set?

When in doubt about how to proceed, construct an example – preferably small.
Let's assume the input is {1, 2} and we take member 2 away.

Set | Subsets
-|-
{1}  |  {}, {1}
{1, 2}  |  {}, {1}, {2}, {1, 2}

We can see that the subsets of {1, 2} are the subsets of {1} plus
the subsets of {1} with the removed member 2 added to them:
{2} is obtained by adding 2 to {} and {1, 2} is obtained by adding 2 to {1}.

More generally, each subset of S $\cup$ {_item_} either doesn't include _item_,
in which case it's a subset of S, or it does include _item_, in which case
it's a subset of S with _item_ added to it.

Here's an algorithm for subsets(_items_),
with _solution_ as the output variable.

1. if _items_ = {}:
   1. let _solution_ be ({})
2. otherwise:
   1. let _item_ be an element of _items_
   2. let _rest_ be _items_ − {_item_}
   1. let _solution_ be subsets(_rest_)
   1. for each _subset_ in _solution_:
      1. append _subset_ $\cup$ {_item_} to _solution_

Step&nbsp;1.1&nbsp;handles the base case, steps 2.2 and 2.3 decrease and recur, and
step&nbsp;2.4 combines the result for the rest of the set with the removed item.

Step&nbsp;2.4.1 is appending to the sequence that is being iterated over.
The sequence is thereby always getting longer and the iteration never stops.
The algorithm is incorrect because it enters an infinite loop,
without ever producing the output.

<div class="alert alert-warning">
<strong>Note:</strong> A loop iterating over a collection must not increase that collection.
</div>

Here's a correct version.

1. if _items_ = {}:
   1. let _solution_ be ({})
2. otherwise:
   1. let _item_ be an element of _items_
   2. let _rest_ be _items_ − {_item_}
   1. let _subsolution_ be subsets(_rest_)
   1. let _new subsets_ be the empty sequence
   1. for each _subset_ in _subsolution_:
      1. append _subset_ $\cup$ {_item_} to _new subsets_
   1. let _solution_ be _subsolution_ concatenated with _new subsets_

Step&nbsp;2.6 puts the new subsets after the recursively generated ones
to keep the order in which they were generated.

Let's see the algorithm in action. Most of the algorithm is straightforward to
translate to code, using Python's [set operators](../08_Unordered/08_4_set.ipynb#8.4-Sets).
Step&nbsp;2.1 requires some thinking. Method `pop`,
introduced in [Section&nbsp;11.2.2](../11_Search/11_2_factorisation.ipynb#11.2.2-Compute-solutions),
removes an arbitrary member from a set, but we shouldn't change the input.
The simplest way I can think of implementing step&nbsp;2.1 is to convert the
set to a list and pick the first item.

In [1]:
def subsets(items: set) -> list:
    """Return all subsets of items, in the order generated."""
    if items == set():      # alternative: len(items) == 0
        return [set()]
    item = list(items)[0]
    rest = items - {item}
    subsolution = subsets(rest)
    new_subsets = []
    for subset in subsolution:
        new_subsets.append(subset.union({item}))
    return subsolution + new_subsets

subsets({1, 2, 3})

[set(), {3}, {2}, {2, 3}, {1}, {1, 3}, {1, 2}, {1, 2, 3}]

The empty subset (the base case) is generated first, then some member is added.
The third and fourth subsets add another member to the first two subsets.
The last four subsets add the remaining third member to the first four subsets:
the fifth subset adds the item to the first (empty) subset,
the sixth subset adds the item to the second subset, and so on.

### 12.7.2 Generating permutations

The next problem is to generate all rearrangements of a given sequence of items.
Once more, I'll output a sequence to see in which order the permutations
are generated.

The smallest input, i.e. the base case, is the empty sequence,
which has a single permutation: itself.

The second question is: if we have the permutations of tail(S),
how do we construct the permutations for sequence S?
Let's consider input sequence (1, 2, 3) with head 1 and tail (2,&nbsp;3).

Sequence | Permutations
-|-
(1, 2, 3)  | (1, 2, 3) (1, 3, 2) (2, 1, 3) (2, 3, 1) (3, 1, 2) (3, 2, 1)
(2, 3)  |  (2, 3) (3, 2)

The permutations for (1, 2, 3) can be obtained by inserting head 1 in
all possible positions of each permutation for (2, 3). For example,
permutation (2, 3) leads to permutations (1, 2, 3), (2,&nbsp;1,&nbsp;3) and (2, 3, 1).

More generally, every permutation P of tail(S) with length _n_ leads to _n_
permutations of S, by inserting head(S) at positions
0, 1, ..., _n_ – 1, and _n_. The last insertion is effectively an append.
Here's an algorithm for permutations(_items_)
with output variable _arrangements_.

1. if _items_ is empty:
   1. let _arrangements_ be the empty sequence
2. otherwise:
   1. let _arrangements_ be the empty sequence
   2. let _item_ be head(_items_)
   2. for each _permutation_ in permutations(tail(_items_)):
      1. for each _index_ from 0 to │*permutation*│:
         1. let _new permutation_ be a copy of _permutation_
         2. insert _item_ in _new permutation_ at position _index_
         2. append _new permutation_ to _arrangements_

The easiest way to copy a list (step&nbsp;2.3.1.1) is to slice all of it.
Here's the code.

In [2]:
%run -i ../m269_rec_list

def permutations(items: list) -> list:
    """Return all permutations of items in the order generated."""
    if len(items) == 0:
        return []
    arrangements = []
    item = head(items)
    for permutation in permutations(tail(items)):
        for index in range(len(permutation) + 1):
            new_permutation = permutation[0:len(permutation)]
            new_permutation.insert(index, item)
            arrangements.append(new_permutation)
    return arrangements

permutations([1, 2, 3])

[]

Hmm, the plot thickens.
Why on Earth isn't there a single permutation?!

This is an actual mistake I did and it took me some minutes to figure it out.
The reason is that `permutations([])` returns the empty list, so
the outer for-loop doesn't execute for `permutations([3])` because
there's no permutation in an empty list. So the function call returns
the empty list and so do `permutations([2, 3])` and `permutations([1, 2, 3])`.
In a nutshell, the algorithm needs the permutations for tails(S) in order to
construct the permutations for S, but there are none.

Can you suggest how to easily fix the problem?
(Hint: see the subset generation algorithm.)

___

The subset generation algorithm doesn't return the empty set for the base case:
it returns a sequence with an empty set. Likewise,
this algorithm must return a sequence with the empty permutation.
Step&nbsp;1.1 must be changed to

1. 1. let _arrangements_ be ( () )

and the second line of code must become `return [ [] ]`.
If you make these changes and run the code again, it will work as expected.

Everyone makes mistakes.
The important thing is to learn from them to avoid repeating them.
Here, I rushed the analysis of the base case. I wrote above:

> The smallest input, i.e. the base case, is the empty sequence,
> which has a single permutation: itself.

That's correct but my failure to answer the second part of the question
'What's the base case and the corresponding output?' led to the mistake.
The moral of the story is to always follow the 'checklist' of questions.

⟵ [Previous section](12_6_avoid_slicing.ipynb) | [Up](12-introduction.ipynb) | [Next section](12_8_multiple.ipynb) ⟶